## Лабораторная работа №4. Построение рекомендательная системы.

### Задание.
***
**Цель лабораторной работы:** изучение разработки рекомендательных моделей.

**Задание:**  
1. Выбрать произвольный набор данных (датасет), предназначенный для построения рекомендательных моделей.
2. Опираясь на [материалы лекции](https://nbviewer.jupyter.org/github/ugapanyuk/ml_course_2021/blob/main/common/notebooks/recom/recom.ipynb), сформировать рекомендации для одного пользователя (объекта) двумя произвольными способами.
3. Сравнить полученные рекомендации (если это возможно, то с применением метрик).

### Выполнение.
***
Задания буду выполнять на [датасете](https://github.com/bhargaviparanjape/clickbait/tree/master/dataset) из статьи "Stop Clickbait: Detecting and Preventing Clickbaits in Online News Media".  
Датасет состоит из 2х файлов (кликбейт и не кликбейт заголовки). Подготовоим единый DataFrame с этими данными:

In [2]:
import re
import nltk
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from nltk import WordPunctTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.metrics.pairwise import cosine_similarity

data_clickbait = pd.read_csv("../data/clickbait_data.txt", sep = "\n\n", engine="python", names = ["header_text"], header = None)
data_non_clickbait = pd.read_csv("../data/non_clickbait_data.txt", sep = "\n\n", engine="python", names = ["header_text"], header = None)
dataset = data_clickbait.append(data_non_clickbait)
dataset = dataset.sample(frac = 1, random_state = 100)
dataset = dataset.reset_index(drop = True)
dataset

,header_text
0,"In Michigan, Bank Lends Little of Its Bailout ..."
1,"Four dead, more than a million in U.S. without..."
2,"In Wyoming, Debate Rages Over Elk Feeding Program"
3,Bryant and Lakers Return to the N.B.A. Finals
4,This Baby's Reaction To Hearing About How She ...
...,...
31995,"When You Binge-Watch ""Making A Murderer"" And T..."
31996,Schiphol airliner crash blamed on altimeter fa...
31997,Radiohead Release Rejected Bond Theme Song And...
31998,Chernobyl Taking a Toll on Invertebrates Too


Выполним токенизацию и стемминг текстов заголовков:

In [5]:
documents = dataset["header_text"].tolist()
nltk.download('stopwords')
stop_words = stopwords.words('english')
word_corpus = list()
tokenizer = WordPunctTokenizer()
stemmer = PorterStemmer()
for line in documents:
    line = re.sub("[^a-z]"," ", line.strip().lower()) # Избавляемся от не a-z символов
    words = tokenizer.tokenize(line)
    words = [stemmer.stem(word) for word in words if not word in stop_words]
    word_corpus.append(words)

[nltk_data] Downloading package stopwords to /home/adminu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Выполним векторизацию заголовков на основе алгоритма word2vec:

In [6]:
%time w2v_model = word2vec.Word2Vec(word_corpus, vector_size = 100, workers = 4, sample=1e-3) #min_count = 10, window=10,
word_vectors = w2v_model.wv # Сохраним "словарь", полученный в результате обучения: "слово:вектор"

CPU times: user 5.34 s, sys: 77.2 ms, total: 5.42 s
Wall time: 3.83 s


Определим рекомендации для такого заголовка:

In [10]:
print(documents[300])
target_header_vec = word_vectors[300]

Fernando Alonso wins 2010 Singapore Grand Prix
